In [47]:
import win32com.client
import pandas as pd
import numpy as np
import datetime as dt
import time
from tqdm import tqdm

In [48]:
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
instStockChart = win32com.client.Dispatch('CpSysDib.CpSvr7254')

In [49]:
# 대신증권 API 연결 확인. 1 출력시 연결 성공, 0 출력시 연결 실패
def connectCheck():
    print('[시스템]:증권사 연결 시도 중')
    if(instCpCybos.IsConnect == 1):
        print('[시스템]:증권사 연결 완료!')

In [50]:
# 증권시장에서 상장되어 있는 종목의 갯수 출력 -> 새로운 종목이 상장되거나 폐지됨에 따라 값이 달라진다.
def canTradeStockCount():
    print('[시스템]:거래 가능한 종목 개수 : '+str(instCpStockCode.GetCount())+'개 체크')

## 데이터 수집

In [51]:
# 대신증권 기준으로 정렬된 상장된(주식거래가 가능한) 종목을 전부 가져오기
def getStockNameCode():
    print('[시스템]:거래 가능한 종목 리스트 목록 가져오는 중')
    code = [] # 종목코드
    name = [] # 종목명

    for i in range(0, instCpStockCode.GetCount()):
        code.append(instCpStockCode.GetData(0, i))
        name.append(instCpStockCode.GetData(1, i))

    codeNameList = pd.DataFrame({
        '종목코드':code
        , '종목명':name
    })
    # codeNameList
    if(len(codeNameList) == 0):
        print('[시스템]:거래 가능한 종목 리스트 목록 가져온 개수 : '+str(len(codeNameList))+'개')
        print('[시스템]:거래 가능한 종목 리스트 목록 가져오기에 문제가 발생하였습니다.')
    else:
        print('[시스템]:거래 가능한 종목 리스트 목록 가져오기 완료!')

    return codeNameList

In [52]:
def jongmokSerach(jongmokSearchList, codeNameList):
    code = []
    name = []
    for names in jongmokSearchList:
        code.append(codeNameList[codeNameList['종목명'] == names]['종목코드'].iloc[0])
        name.append(codeNameList[codeNameList['종목명'] == names]['종목명'].iloc[0])

    selCodeNameList = pd.DataFrame({
        '종목코드':code
        , '종목명':name
    })
    return selCodeNameList

In [53]:
def getjusik(jusikCode, jusikName, start, end):
    pdListed = []

    instStockChart.SetInputValue(0, jusikCode)
    instStockChart.SetInputValue(1, 0)
    instStockChart.SetInputValue(2, start)
    instStockChart.SetInputValue(3, end)
    instStockChart.SetInputValue(4, ord('0'))
    instStockChart.SetInputValue(5, 0)
    instStockChart.SetInputValue(6, ord('1'))

    instStockChart.BlockRequest()

    num = instStockChart.GetHeaderValue(1)
                                                            
    for i in range(num):
        Listed = {}
        Listed['종목코드'] = jusikCode                             # 종목코드
        Listed['종목명'] = jusikName                               # 종목명
        Listed['날짜'] = instStockChart.GetDataValue(0, i)        # 날짜
        Listed['개인'] = instStockChart.GetDataValue(1, i)        # 개인
        Listed['외국인'] = instStockChart.GetDataValue(2, i)      # 외국인
        Listed['기관계'] = instStockChart.GetDataValue(3, i)      # 기관계
        Listed['금융투자'] = instStockChart.GetDataValue(4, i)    # 금융투자
        Listed['보험'] = instStockChart.GetDataValue(5, i)        # 보험
        Listed['투신'] = instStockChart.GetDataValue(6, i)        # 투신
        Listed['은행'] = instStockChart.GetDataValue(7, i)        # 은행
        Listed['기타금융'] = instStockChart.GetDataValue(8, i)    # 기타금융
        Listed['연기금'] = instStockChart.GetDataValue(9, i)      # 연기금
        Listed['기타법인'] = instStockChart.GetDataValue(10, i)   # 기타법인
        Listed['기타외인'] = instStockChart.GetDataValue(11, i)   # 기타외인
        Listed['사모펀드'] = instStockChart.GetDataValue(12, i)   # 사모펀드

        pdListed.append(Listed)

    dataDf = pd.DataFrame(pdListed)
    return dataDf

In [54]:
def aaa(stockNameList):
    return pd.read_csv('./stockdata/'+stockNameList[0]+' 주식 데이터(투자주체별).csv', encoding='euc-kr')

In [55]:
# 수신 받은 주식데이터(투자주체별) 정렬하기
def reverse(stockNameList):
    print('[시스템]:수신받은 데이터 날짜, 시간순으로 정렬 작업중')
    for i in range(len(stockNameList)):
        data1 = pd.read_csv('./stockdata/'+stockNameList[i]+' 주식 데이터(투자주체별).csv', encoding='euc-kr')

        data1.sort_values(['날짜'], inplace=True)
        data1.reset_index(inplace=True)
        data1.drop('index', axis=1, inplace=True)
        data1.to_csv('./stockdata/'+stockNameList[i]+' 주식 데이터(투자주체별).csv', encoding='euc-kr', index=False)
    print('[시스템]:정렬 완료!')

In [56]:
stockNameList = ['SK하이닉스'] # 이곳에 수신 받고 싶은 종목 이름을 정확히 입력해주세요. 여러개 가능.

connectCheck()
canTradeStockCount()
codeNameList = getStockNameCode()
selCodeNameList = jongmokSerach(stockNameList, codeNameList)

a = dt.datetime(2021,6,10) # 수신 받고 싶은 원하는 시작 날짜를 입력하세요.
b = dt.datetime(2023,6,23) # 수신 받고 싶은 원하는 종료 날짜를 입력하세요.
number = b-a

for j in range(0, selCodeNameList.shape[0]):
    print('[시스템]:'+str(a.strftime('%Y%m%d'))+' ~ '+str(b.strftime('%Y%m%d'))+' 사이에 거래된 '+str(selCodeNameList['종목명'].iloc[j])+' 종목 주가 데이터(투자주체별) 가져오는 중')
    frame = []
    for i in tqdm(range(0, number.days, 2)):
        c = a + dt.timedelta(days=i)
        d = a + dt.timedelta(days=i+1)
        result = getjusik(selCodeNameList['종목코드'].iloc[j], selCodeNameList['종목명'].iloc[j], c.strftime('%Y%m%d'), d.strftime('%Y%m%d'))
        frame.append(result)
        time.sleep(0.3)
    hapdata = pd.concat(frame)
    hapdata.to_csv('./stockdata/'+selCodeNameList['종목명'].iloc[j]+' 주식 데이터(투자주체별).csv', encoding='euc-kr', index=False)
    print('[시스템]:'+selCodeNameList['종목명'].iloc[j]+' 종목의 주식 데이터(투자주체별) 요청 수신성공 및 csv로 저장 성공!')

reverse(stockNameList)
print('[시스템]:수신요청 종료')
aaa(stockNameList)

[시스템]:증권사 연결 시도 중
[시스템]:증권사 연결 완료!
[시스템]:거래 가능한 종목 개수 : 3969개 체크
[시스템]:거래 가능한 종목 리스트 목록 가져오는 중
[시스템]:거래 가능한 종목 리스트 목록 가져오기 완료!
[시스템]:20210610 ~ 20230623 사이에 거래된 SK하이닉스 종목 주가 데이터(투자주체별) 가져오는 중


  0%|          | 0/372 [00:00<?, ?it/s]

100%|██████████| 372/372 [01:56<00:00,  3.18it/s]

[시스템]:SK하이닉스 종목의 주식 데이터(투자주체별) 요청 수신성공 및 csv로 저장 성공!
[시스템]:수신받은 데이터 날짜, 시간순으로 정렬 작업중
[시스템]:정렬 완료!
[시스템]:수신요청 종료


,종목코드,종목명,날짜,개인,외국인,기관계,금융투자,보험,투신,은행,기타금융,연기금,기타법인,기타외인,사모펀드
0,A000660,SK하이닉스,20220623,131309,-783362,637219,654235,-14259,23074,13651,2861,-63132,14919,-85,20789
1,A000660,SK하이닉스,20220624,9741,-976492,959927,1042101,-14090,35497,19456,966,-239951,6329,495,115948
2,A000660,SK하이닉스,20220627,-525143,729606,-201309,-213117,9382,64610,7224,118,-144503,-3250,96,74977
3,A000660,SK하이닉스,20220628,-454835,810547,-370661,-254380,-1253,58512,7721,1118,-247350,14128,821,64971
4,A000660,SK하이닉스,20220629,76313,75794,-176522,-187544,20020,25746,-1769,-1649,-34912,23990,425,3586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,A000660,SK하이닉스,20230619,666917,-337603,-320867,415799,-55706,-121891,17575,-23782,-67905,-9816,1369,-484957
245,A000660,SK하이닉스,20230620,-223358,172460,69592,127926,30005,13856,-968,4688,75415,-17605,-1089,-181330
246,A000660,SK하이닉스,20230621,-156080,119203,121122,196380,49334,9748,-282,14688,36089,-84760,515,-184835
247,A000660,SK하이닉스,20230622,176227,-383299,195721,111289,-14535,-15329,3533,2251,75583,7993,3358,32929
